In [ ]:
# Imports
from dotenv import load_dotenv
import pyfabricops as pf
import os
from pathlib import Path


# Setups
pf.set_auth_provider('env')
pf.setup_logging(format_style='minimal', level='info') 

In [ ]:
# Find root path
def find_root_path(
    start: Path = None,
    markers: list[str] = None
) -> Path:
    start = (start or Path.cwd()).resolve()
    markers = markers or [
        ".git",         
        ".gitignore",
    ]
    for directory in [start, *start.parents]:
        for m in markers:
            if (directory / m).exists():
                return directory
    return start

root_path = find_root_path()
print(f"Root path: {root_path}")

In [ ]:
# Capture git info
branches_path = Path(root_path) / 'branches.json'
branch = pf.get_current_branch()
workspace_suffix = pf.get_workspace_suffix(branch, path = branches_path) 
print('Branch:', branch, 'Workspace Suffix:', workspace_suffix) 

In [ ]:
# Project config
project = 'PF_002_Fabric'
project_path = Path(root_path)/'src'
workspace_alias ='PF_002_Fabric'
workspace_name = workspace_alias + workspace_suffix
capacity = '7732a1eb-3893-4642-a85c-93fc3f35d076' 
print('Workspace name: ', workspace_name) 

In [ ]:
workspaces_roles_path = Path(root_path) / 'workspaces_roles.json'
roles = pf.read_json(workspaces_roles_path) 
display(pf.json_to_df(roles))  

In [ ]:
# Create the workspace

pf.create_workspace(
    workspace_name,
    description='CI/CD with Microsoft Fabric and PyFabricOps',
    capacity=capacity,
    roles=roles,
    df=True,
)

In [ ]:
# Create project
pf.export_workspace_config(
    workspace_name,
    project_path,
    branches_path=branches_path,
    workspace_suffix=workspace_suffix
) 

In [ ]:
# Create `storage` folder
storage_folder = pf.create_folder(
    workspace_name,
    'Storage',
)
storage_folder_id = storage_folder['id'] 
display(pf.json_to_df(storage_folder))
print('Folder `storage` created with id: ', storage_folder_id) 

In [ ]:
# Create `Orchestration` folder
orchestration_folder = pf.create_folder(
    workspace_name,
    'Orchestration',
)
orchestration_folder_id = orchestration_folder['id'] 
display(pf.json_to_df(orchestration_folder))
print('Folder `Orchestration` created with id: ', orchestration_folder_id) 

In [ ]:
# Create `PowerBI` folder
powerbi_folder = pf.create_folder(
    workspace_name,
    'PowerBI',
)
powerbi_folder_id = powerbi_folder['id'] 
display(pf.json_to_df(powerbi_folder))
print('Folder `PowerBI` created with id: ', powerbi_folder_id) 

In [ ]:
# Create subfolders `DirectLake` and `Import` 
subfolders = ['DirectLake', 'Import']
parent_folder_id = powerbi_folder_id 
for subfolder in subfolders:
    subfolder_created = pf.create_folder(
        workspace_name, 
        subfolder, 
        parent_folder=powerbi_folder_id,
    )
    subfolder_created_id = subfolder_created['id']
    print(f'Folder `{subfolder}` created with id: {subfolder_created_id} under `PowerBI`')  


In [ ]:
# Export folders to config
pf.export_folders(
    workspace_name,
    project_path,
    branches_path=branches_path,
    workspace_suffix=workspace_suffix
) 

In [ ]:
# Load config
config_path = Path(project_path)/'config.json'
config = pf.read_json(config_path)

In [ ]:
workspace_id = config[branch][workspace_alias]['workspace_config']['workspace_id']
storage_folder_id_c = config[branch][workspace_alias]['folders']['Storage'] 

In [ ]:
# Create lakehouse
pf.create_lakehouse(
    workspace_id,
    'MainStorage', 
    description='The main storage of the lakehouse',  
    folder=storage_folder_id_c,
    df=True
)

In [ ]:
# Export lakehouse to config
pf.export_lakehouse(
    workspace_id,
    'MainStorage',
    project_path,
    workspace_path=workspace_alias,
    branches_path=branches_path,
    workspace_suffix=workspace_suffix,
)

In [ ]:
# Create warehouse
pf.create_warehouse(
    workspace_id,
    'MainWarehouse', 
    description='The SQL instance for the project',  
    folder=storage_folder_id_c,
    df=True
)

In [ ]:
# Export warehouse to config
pf.export_warehouse(
    workspace_id,
    'MainWarehouse',
    project_path,
    workspace_path=workspace_alias,
    branches_path=branches_path,
    workspace_suffix=workspace_suffix,
)

In [ ]:
# Create GitHub connection

from pyfabricops import get_connection


load_dotenv() 
github_token = os.getenv('GH_TOKEN') 

github_connection = pf.create_github_source_control_connection(
    'PF_002_Fabric',
    'https://github.com/alisonpezzott/pyfabricops-demo-002-fabric',
    github_token,
) 

if github_connection:
    github_connection_id = github_connection['id']
else:
    github_connection_id = get_connection('PF_002_Fabric')['id'] 

print('GitHub Connection: ', github_connection_id) 


In [ ]:
connections_roles_path = Path(root_path) / 'connections_roles.json'
connections_roles = pf.read_json(connections_roles_path) 
display(pf.json_to_df(connections_roles))  

In [ ]:
# Add roles to connection
roles_added = []
for role in connections_roles:
    role_added = pf.add_connection_role_assignment(
        github_connection_id, 
        role['user_uuid'], 
        role['user_type'], 
        role['role']
    )
    if role_added:
        roles_added.append(role_added)

if roles_added:
    print('Added roles')
    display(pf.json_to_df(roles_added)) 
else:
    print('Current roles')
    current_connection_roles = pf.list_connection_role_assignments(github_connection_id, df=True) 
    display(current_connection_roles) 

In [ ]:
# Connect workspace to git repository
pf.github_connect(
    workspace_id,
    github_connection_id,
    owner_name='alisonpezzott',
    repository_name='pyfabricops-demo-002-fabric',
    branch_name=branch,
    directory_name='src/' + workspace_alias
)


In [ ]:
# Init Git
pf.git_init(workspace_id) 

In [ ]:
# Commit to git
pf.commit_to_git(
    workspace_id,
    df=True,
)

In [ ]:
# Check status
pf.git_status(workspace_id) 